## Preparation

1. Enable APIs (Run on Cloud Shell)
```bash
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com \
  calendar-json.googleapis.com
```

2. Create Vertex AI Workbench instance (Run on Cloud Shell)
```bash
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

3. [Configure the OAuth consent screen and choose scopes](https://developers.google.com/workspace/guides/configure-oauth-consent)
  - Select Internal user type.
 
4. [Configure OAuth client ID credentials](https://developers.google.com/workspace/guides/create-credentials#oauth-client-id)
  - Create "Web application" client and add `https://www.example.com` to the authorized redirect URI.
  - Download the client secret JSON file as `credentials.json`

5. Navigate to [Workbench Instances](https://console.cloud.google.com/vertex-ai/workbench/instances) on Cloud Console and open [JupyterLab].
  - Open a new "Python 3(ipykernel)" notebook.
  - Upload `credentials.json` to the current directory.
  
Now you can follow the notebook contents below.

## Install packages

In [ ]:
%pip install --upgrade --user \
    google-api-python-client \
    google-auth-httplib2 google-auth-oauthlib \
    google-genai \
    google-adk

In [ ]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Define tools

In [1]:
import os, json, pprint, datetime, uuid
import vertexai
from google import genai
from google.genai.types import (
    HttpOptions, GenerateContentConfig,
    Part, UserContent, ModelContent
)

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
!curl -OL https://github.com/enakai00/adk_examples/raw/main/misc/event_sample.pdf
!gsutil cp event_sample.pdf gs://{PROJECT_ID}/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  386k  100  386k    0     0  1342k      0 --:--:-- --:--:-- --:--:-- 6238k
Copying file://event_sample.pdf [Content-Type=application/pdf]...
/ [1 files][386.8 KiB/386.8 KiB]                                                
Operation completed over 1 objects/386.8 KiB.                                    


### Tool to extract event info from PDF

In [3]:
def generate_response(system_instruction, contents,
                      response_schema, temperature=0.4,
                      model='gemini-2.0-flash-001'):
    client = genai.Client(vertexai=True,
                          project=PROJECT_ID, location=LOCATION,
                          http_options=HttpOptions(api_version='v1'))
    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            temperature=temperature,
            response_mime_type='application/json',
            response_schema=response_schema,
        )
    )
    return '\n'.join(
        [p.text for p in response.candidates[0].content.parts if p.text]
    )

In [4]:
def _get_event_from_pdf(file_name):
    
    file_uri = f'gs://{PROJECT_ID}/{file_name}'

    response_schema = {
        "type": "object",
        "properties": {
            "title":  {"type": "string"},
            "date_time": {"type": "string"},
            "duration_minutes":  {"type": "string"},
            "location": {"type": "string"},
            },
        "required": ["title", "date_time", "duration_minutes", "location"],
    }
    system_instruction = '''
Extract event information from PDF file.
try to add as much information as possible to title.
data_time format is '%Y-%m-%dT%H:%M:%S%z'.
If duration_minutes is not available set it "unknown" as default.
If lacation is not available set it "unknown" as default.
'''

    parts = []
    parts.append(Part.from_text(text='[PDF]'))
    parts.append(Part.from_uri(file_uri=file_uri, mime_type='application/pdf'))
    contents=[UserContent(parts=parts)]

    return generate_response(system_instruction, contents,
                             response_schema, temperature=0.2)


def get_event_from_pdf(file_name: str) -> dict:
    """
    Extract event information from PDF file.
   
    Args:
        file_name: file name of the PDF

    Returns:
        dict: A dictionary containing the event information
            title: Tile of the event
            date_time: Start data/time of the event in '%Y-%m-%dT%H:%M:%S%z'
            duration_minutes: Time duration of the event in minutes
            location: Location of the event
    """
    response = _get_event_from_pdf(file_name)
    return json.loads(response)

In [5]:
get_event_from_pdf('event_sample.pdf')

{'date_time': '2025-06-13T19:00:00+09:00',
 'duration_minutes': '120',
 'location': 'Google 渋谷オフィス',
 'title': 'GDG Tokyo Build with AI: Python x Gemini 2.0 API ハンズオン'}

### Google login

In [6]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from urllib.parse import urlparse, parse_qs

def google_login():
    SCOPES = ["https://www.googleapis.com/auth/calendar"]
    creds = None    
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES, redirect_uri='https://www.example.com',
    )
    auth_url, _ = flow.authorization_url(prompt='consent')
    print('Please open the following URL in your browser and authorize the application:')
    print(auth_url)
    url = input('Enter the URL of the redirected page here: ')
    parsed_url = urlparse(url)
    code = parse_qs(parsed_url.query)['code'][0]
    flow.fetch_token(code=code)
    creds = flow.credentials
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
    print('Access token has been stored in "token.json".')

In [ ]:
_ = google_login()

### Tool to interact with Google Calendar

In [7]:
def get_current_datetime():
    """
    Get the current date/time
   
    Args: None
        
    Returns:
        dict: A dictionary containing the current date/time in '%Y-%m-%dT%H:%M:%S%z'
    """
    now = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%dT%H:%M:%S%z')
    return {'now': now}

In [8]:
def _get_upcoming_events():
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
    try:
        service = build('calendar', 'v3', credentials=creds)
        now = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%dT%H:%M:%S%z')
        now = now[:-2] + ':' + now[-2:] # Format to include colon in timezone offset
        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=now,
                maxResults=10,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])
        result = []
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            end = event["end"].get("dateTime", event["start"].get("date"))
            if 'attendees' in event.keys():
                attendees = [item.get('email') for item in event['attendees']]
            else:
                attendees = []
            result.append({
                'start': start,
                'end': end,
                'attendees': attendees,
                'event': event['summary']
            })
        return {'upcoming_events': result}

    except HttpError as error:
        return {'error': error}
        

def get_upcoming_events() -> dict:
    """
    Get upcoming events in Google Calendar
   
    Args: None
        
    Returns:
        dict: A dictionary where the 'upcoming_events' element contains
              a list of upcoming events in Google Calendar.
              Each item in the list is a dict with the following keys.
            - start: Start data/time of the event
            - end: end data/time of the event
            - attendees: list of attendees
            - event: Summary of the event
    """
    response = _get_upcoming_events()
    return response

In [9]:
get_upcoming_events()

{'upcoming_events': [{'start': '2025-05-12T14:00:00+09:00',
   'end': '2025-05-12T15:00:00+09:00',
   'attendees': ['etsuji@example.com'],
   'event': 'meeting with etsuji'},
  {'start': '2025-05-12T19:00:00+09:00',
   'end': '2025-05-12T21:00:00+09:00',
   'attendees': [],
   'event': '誕生日会'}]}

In [10]:
def _create_event(start_datetime, end_datetime, title, attendees):
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
    try:
        service = build("calendar", "v3", credentials=creds)
        event = {
            'summary': title,
            'start': {
                'dateTime': start_datetime,
            },
            'end': {
                'dateTime': end_datetime,
            },
            'attendees': [{'email': item} for item in attendees],
        }
        event = service.events().insert(calendarId='primary', body=event).execute()
        return {'htmlLink': event.get('htmlLink')}

    except HttpError as error:
        return {'error': error}
    
def create_event(start_datetime: str, end_datetime: str, title: str, attendees: list) -> dict:
    """
    Create a calendar event
   
    Args:
        start_datetime: start date/time of the event in '%Y-%m-%dT%H:%M:%S%z'
        end_datetime: end date/time of the event in '%Y-%m-%dT%H:%M:%S%z'
        title: title of the event
        attendees: list of attendees' email address
        
    Returns:
        dict: A dictionary containing the event link.
    """
    response = _create_event(start_datetime, end_datetime, title, attendees)
    return response

## Calendar Agent

In [11]:
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

In [12]:
instruction = """
    You are an agent who handles user's google calendar events.
    Your entire output should be in Japanese. No markdowns.
    
    **General conditions**
    - Timezone is JST.
    - When the user mentions relative date/time such as today, tomorrow, etc., don't assume the date/time yourself.
      Instead, use get_current_datetime() first to know the current data/time,
      then convert the relative date/time into absolute date/time based on the current date/time.
    
    **Interaction flow:**

    1. Queries about upcoming events:
        * If the user asks about upcoming events,
          get calendar events using get_upcoming_events() to handle the query.
          - Show event information in a well-structured human-readable ascii table format.
            date/time format is '%Y-%m-%d %H:%M:%S'. Header is in Japanese.
          
    2. Queries about PDF file:
        * If the user asks about PDF file with specifying the filename,
          use get_event_from_pdf() to extract event information from the PDF to handle the query.
         
    3. Create a calendar event:
        * If the user asks to create a new calendar event, follow the steps below:
          - Use get_current_datetime() to know the current date/time.
          - Collect information about the event such as start date/time, end date/time (or duration),
            title, attendees' email address)
          - Show the event information to be created in a well-strctured human-readable bullet list format.
          - Get an approval from the user and use create_events() to create an event in the calendar,
            then show the event url in HTML such as <a href="__URL__">リンク</a>
"""

In [13]:
calendar_agent = LlmAgent(
    model='gemini-2.0-flash-001',
    name='calendar_client_agent',
    description=(
        'This agent handles queries regarding calendar events.'
    ),
    instruction=instruction,
    tools=[
        get_current_datetime,
        get_upcoming_events,
        get_event_from_pdf,
        create_event,
    ],
)

In [14]:
class LocalApp:
    def __init__(self, agent, user_id='default_user'):
        self._agent = agent
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._agent.name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._agent.name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        agent_name = None
        async for event in async_events:
            if DEBUG:
                print(f'----\n{event}\n----')
            if (event.content and event.content.parts):
                response = ''
                for p in event.content.parts:
                    if p.text:
                        response += f'[{event.author}]\n\n{p.text}\n'
                if response:
                    print(response)
                    result.append(response)
        return result

## Test run

In [15]:
client = LocalApp(calendar_agent)
DEBUG = False

In [16]:
query = '''
何ができますか？
'''

_ = await client.stream(query)

[calendar_client_agent]

カレンダーイベントに関するお問い合わせに対応できます。例えば、今後のイベントの確認、PDFファイルからのイベント情報の抽出、カレンダーイベントの作成などです。



In [17]:
query = '''
この後の予定は？
'''

_ = await client.stream(query)

[calendar_client_agent]

今後の予定は以下の通りです。

| 開始日時            | 終了日時            | イベント名           | 参加者                      |
| ------------------- | ------------------- | ----------------- | ------------------------- |
| 2025-05-12 14:00:00 | 2025-05-12 15:00:00 | meeting with etsuji | etsuji@example.com        |
| 2025-05-12 19:00:00 | 2025-05-12 21:00:00 | 誕生日会             |                           |



In [18]:
query = '''
明日、１２時から３０分、田中さんと Agentspace の打ち合わせを入れて。
メアドは tanaka@example.com
'''

_ = await client.stream(query)

[calendar_client_agent]

明日の12:00から30分、田中さんと Agentspace の打ち合わせですね。以下の内容でよろしいでしょうか？

*   タイトル: Agentspace の打ち合わせ
*   開始日時: 2025-05-13 12:00:00
*   終了日時: 2025-05-13 12:30:00
*   参加者: tanaka@example.com

よろしければ「はい」と、修正が必要な場合は修正内容をお知らせください。




In [19]:
query = '''
はい。
'''
_ = await client.stream(query)

[calendar_client_agent]

カレンダーにイベントを作成します。


[calendar_client_agent]

イベントを作成しました。

<a href="https://www.google.com/calendar/event?eid=cHRsbTE5czBpbHR0NWE2OGljaGZqaGY1ZDggYWRtaW5AZW5ha2FpLmFsdG9zdHJhdC5jb20">リンク</a>




In [20]:
query = '''
ファイル event_sample.pdf のイベントに参加します。参加者は、enakai@example.com
'''

_ = await client.stream(query)

[calendar_client_agent]

event_sample.pdf からイベント情報を抽出します。


[calendar_client_agent]

ファイル event_sample.pdf から抽出されたイベント情報は以下の通りです。

*   タイトル: Python x Gemini 2.0 API ハンズオン
*   開始日時: 2025-06-13 19:00:00
*   終了日時: 2025-06-13 21:00:00
*   場所: Google 渋谷オフィス
*   参加者: enakai@example.com

このイベントに参加しますか？よろしければ「はい」と、修正が必要な場合は修正内容をお知らせください。




In [21]:
query = '''
はい。
'''
_ = await client.stream(query)

[calendar_client_agent]

カレンダーにイベントを作成します。


[calendar_client_agent]

イベントを作成しました。

<a href="https://www.google.com/calendar/event?eid=aGJ2bWs1OThnaWpxNWJlNHNpdWNqbjJvNXMgYWRtaW5AZW5ha2FpLmFsdG9zdHJhdC5jb20">リンク</a>


